# 딥러닝을 위한 PyTorch 활용버
## Chapter 3. Convolution Neural Network(CNN)

### Layer
- Image size I
- Kernel size K
- stride S(한 칸 움직이는 거리)
- Output size O
![output size 식, convolution 식](convolution_formula2.png)

### Kernel(커널 혹은 Filter)
- 여러 필터를 이용하여 다양한 모양을 학습
- 각 필터마다 특징을 뽑아낼 수 있음
- 필터가 많을수록 다양한 feature(complex)를 추출할 수 있음
- 필터는 랜덤으로, 필터의 값 또한 학습된다.

### 축소(Subsampling)
- Max pooling : 강한 자극만 기억(물체의 유무, classification 유리)
- Average pooling : 평균적 자극을 기억(전체적인 수치)

### 손실(Loss function)
- Cross Entropy Loss : 잘못된 예측에 대해 패널티를 많이 주기 때문에 주로 이 매트릭을 쓴다

## CNN
- MNIST 데이터 사용 (28x28)

## Import

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [117]:
batch_size = 128
learning_rate = 0.0002
num_epoch = 2

# GPU 사용하기
is_gpu = False

## Data

In [118]:
# 데이터 다운로드
mnist_train = dset.MNIST("./", train = True, transform = transforms.ToTensor(), target_transform = None, download = True)
mnist_test = dset.MNIST("./", train = True, transform = transforms.ToTensor(), target_transform = None, download = True)

In [119]:
# 데이터 확인
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 60000


In [63]:
# 데이터 셋팅
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = batch_size, shuffle = False)

## CNN Model

In [122]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            #nn.Conv2d(128, 256, 3, 1, 1),
            #nn.BatchNorm2d(256),
            #nn.ReLU(),
            #nn.Conv2d(256, 512, 3, 1, 1),
            #nn.BatchNorm2d(512),
            #nn.ReLU(),
            #nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(128*14*14, 400),
            nn.ReLU(),
            nn.Linear(400, 10)
        )
    def forward(self, x):
            out = self.layer(x)
            out = out.view(batch_size, -1)
            out = self.fc_layer(out)
            return out

model = CNN()

# GPU 사용하기            
if is_gpu:
    model = model.cuda()

## Loss func & Optimizer

In [123]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

## Train

In [124]:
loss_arr = []

for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        optimizer.zero_grad()
        
        if is_gpu:
            x = Variable(image).cuda()
            y = Variable(label).cuda()
        else:
            x = Variable(image)
            y = Variable(label)
        
        output = model.forward(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)
        loss_arr.append(loss)

tensor(2.3034, grad_fn=<NllLossBackward>)


RuntimeError: size mismatch, m1: [128 x 18816], m2: [25088 x 400] at c:\programdata\miniconda3\conda-bld\pytorch_1533090623466\work\aten\src\th\generic/THTensorMath.cpp:2070

## 이거 자꾸 사이즈 오류 발생하는데 아직 이유를 모르겠음....